In [1]:
!pip install kaggle

In [2]:
import os

# Create a .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Move the kaggle.json file to the .kaggle directory
!mv kaggle.json ~/.kaggle/

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API key configured successfully.")

Kaggle API key configured successfully.


In [3]:
import os

# Download the dataset using Kaggle API
!kaggle datasets download -d jangedoo/utkface-new -p .

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 96% 319M/331M [00:00<00:00, 557MB/s]
100% 331M/331M [00:00<00:00, 558MB/s]


In [4]:
import zipfile
import os

# Unzip the downloaded dataset
with zipfile.ZipFile('utkface-new.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

print('Dataset unzipped successfully.')

Dataset unzipped successfully.


In [41]:
age=[]
gender=[]
img_path=[]
for file in os.listdir('/content/utkface_aligned_cropped/UTKFace'):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)


- Here in the image name we can see the first part before _ was the age and the second part was the gender label 0 and 1 so we did split and created a dataframe

In [42]:
len(age)

23708

In [43]:
import pandas as pd
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [44]:
df.head()

,age,gender,img
0,44,1,44_1_1_20170113011544238.jpg.chip.jpg
1,60,0,60_0_2_20170116192628656.jpg.chip.jpg
2,10,1,10_1_0_20170109205208368.jpg.chip.jpg
3,26,1,26_1_4_20161221193707668.jpg.chip.jpg
4,43,1,43_1_0_20170109010025945.jpg.chip.jpg


In [45]:
df.shape

(23708, 3)

In [46]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

-  This parameter specifies the fraction of rows to return from the sample. When frac=1, it means you are taking a sample that includes 100% of the original DataFrame's rows. Essentially, df.sample(frac=1) shuffles all rows of the DataFrame.

- This parameter is used to seed the random number generator. When you set random_state to an integer (like 0 in this case), the shuffling operation will produce the exact same result every time you run the code.


In [47]:
train_df.shape

(20000, 3)

In [48]:
test_df.shape

(3708, 3)

In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [50]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory='/content/utkface_aligned_cropped/UTKFace',
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory='/content/utkface_aligned_cropped/UTKFace',
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [51]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [52]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [53]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [57]:
model = Model(inputs=resnet.input,outputs=(output1,output2))

In [ ]:
from keras.utils import plot_model
plot_model(model)

#  Remeber in the Model the outputs are always in tupple, if passed in list then error will occur, since tensorflow won't accept it

In [58]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
   7642/Unknown 2555s 326ms/step - age_loss: 15.0563 - age_mae: 15.0563 - gender_accuracy: 0.5204 - gender_loss: 0.8106 - loss: 95.3072

# Testing our model

In [1]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os

# Select a random image from the test set for prediction
sample_image_name = test_df.sample(1).iloc[0]['img']
sample_image_path = os.path.join('/content/utkface_aligned_cropped/UTKFace', sample_image_name)

# Load the image
img = image.load_img(sample_image_path, target_size=(200, 200))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) # Create a batch
img_array = img_array / 255.0  # Rescale the image like training data

# Make prediction
predictions = model.predict(img_array)
predicted_age = int(predictions[0][0][0])
predicted_gender_raw = predictions[1][0][0]

# Interpret gender prediction (sigmoid output)
predicted_gender = 'Male' if predicted_gender_raw > 0.5 else 'Female'

# Display the image and predictions
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title(f"Predicted Age: {predicted_age}, Predicted Gender: {predicted_gender}")
plt.axis('off')
plt.show()

NameError: name 'test_df' is not defined